# Домашняя работа №3. «Коллаборативная фильтрация».
## ПАКЕТ SURPRISE

## - используйте данные MovieLens 1M
## - можно использовать любые модели из пакета
## - получите RMSE на тестовом сете 0.87 и ниже

In [1]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.evaluate import GridSearch
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
import numpy as np
from tqdm import tqdm_notebook

import pandas as pd

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [6]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [9]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [59]:
train, test = train_test_split(data, test_size=.15)

## Возьмем разные модели, кот. основаны на методе KNN: KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline.

## С помощью GridSearchCV подберем наилучшие параметры. 

In [11]:
param_grid = {
    'sim_options' : {
                    'name': ['pearson_baseline', 'cosine', 'msd'],
                    'user_based': [False, True]
    },
}
score = {}
for model in  tqdm_notebook([KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline]):
    gsearch = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=2, n_jobs=3)
    gsearch.fit(data)
    score[model] = [gsearch.best_score['rmse'], gsearch.best_params]

In [32]:
num = np.argmin(np.array(list(score.values()))[:, 0])

In [33]:
model = list(score.keys())[num]

In [34]:
name = list(score.values())[num][1]['rmse']['sim_options']['name']
user_based = list(score.values())[num][1]['rmse']['sim_options']['user_based']

In [41]:
algo = model(k=150, sim_options={'name': name, 'user_based': user_based})
algo.fit(train)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [67]:
model

surprise.prediction_algorithms.knns.KNNBaseline

In [42]:
test_pred = algo.test(test)

In [43]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8630


0.8629945606191872

## Т.о. у нас получилось RMSE<0.87 с моделью KNNBaseline.

In [49]:
from surprise import SVD, SVDpp

## Теперь попробуем модели другого типа: SVD, SVD++.

In [53]:
svd_param_grid = {'n_epochs': [20, 25], 
                  'lr_all': [0.007, 0.009, 0.01],
                  'reg_all': [0.4, 0.6]}

svdpp_gs = GridSearchCV(SVDpp, svd_param_grid, measures=['rmse'], cv=3, n_jobs=-1)
svdpp_gs.fit(data)

svd_gs = GridSearchCV(SVD, svd_param_grid, measures=['rmse'], cv=3, n_jobs=-1)
svd_gs.fit(data)

In [60]:
algo = SVDpp(n_epochs=20, reg_all=.2, lr_all=0.007)
algo.fit(train)
test_pred = algo.test(test)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8558


0.855784545236449

In [65]:
algo = SVD(n_epochs=20, lr_all=0.007, reg_all=0.2)
algo.fit(train)
test_pred = algo.test(test)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8679


0.8679267125199513

## Таким образом, наилучшая модель - SVD++ с RMSE = 0.8558